In [20]:
from sqlalchemy import create_engine

# Import pandas, numpy, matplotlib, seaborn libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import time

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics

In [21]:
# URI = 'database-dublin-bus.cnhytboldqes.eu-west-1.rds.amazonaws.com'
# PORT = "3306"
# DB = "database-dublin-bus"
# USER = "group14"
# PASSWARD = "gRouP1422"

# engine = create_engine("mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWARD, URI, PORT, DB))

In [22]:
# df_GTFS = pd.read_sql_table("gtfsr", engine, parse_dates="True", index_col=0)

In [23]:
# df_GTFS.to_csv('GTFS_08_08.csv',index=False)

In [24]:
df_GTFS = pd.read_csv('GTFS_08_08.csv')

In [25]:
df_GTFS

,Time_of_db_update,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay
0,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,2,8220DB000341,120,120
1,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,3,8220B111761,60,60
2,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,4,8220B100421,180,180
3,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,5,8220B1021601,300,300
4,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,10,8250DB000435,240,240
...,...,...,...,...,...,...,...,...,...
641153,1660212000,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,20220811,48,8380B246981,0,0
641154,1660212000,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,20220811,12,8440B352481,0,0
641155,1660212000,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,20220811,11,8220B1353501,0,0
641156,1660212000,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,20220811,48,8380B246981,0,0


In [26]:
#The following is necessary data cleaning process

In [27]:
#First we will apply cleaning process on the collected GTFS data
#delete duplicated rows
df_GTFS = df_GTFS.drop_duplicates()

#Classify the columns type
categorical_columns = df_GTFS[['TripId','RouteId','StopId']].columns
continuous_columns = df_GTFS[['Arr_delay','Dep_delay','StopSequence']].columns


for column in categorical_columns:
    df_GTFS[column] = df_GTFS[column].astype('category')  
    
for column in continuous_columns:
    df_GTFS[column] = df_GTFS[column].astype('int64')
    
df_GTFS['StartDate'] = df_GTFS['StartDate'].astype('str')
df_GTFS['StartTime'] = df_GTFS['StartTime'].astype('str')

#Convert the column 'StartDate' into proper form
df_GTFS['StartDate']=df_GTFS['StartDate'].str[:4] + '-' + df_GTFS['StartDate'].str[4:]
df_GTFS['StartDate']=df_GTFS['StartDate'].str[:7] + '-' + df_GTFS['StartDate'].str[7:]

#Concatenate the date with the time
df_GTFS['StartDate'] = df_GTFS['StartDate'] + str(' ') + df_GTFS['StartTime']
df_GTFS['StartDate'] = pd.to_datetime(df_GTFS['StartDate']).astype('datetime64')

#Extract the line id from the trip_id
df_temporary = df_GTFS['RouteId'].str.split('-',n=2,expand=True)
df_GTFS['LineId'] = df_temporary[1]

#Remove the data of LineId which are in the 42_route list
lineid_list_42routes = ['77A', '16', '40D', '142', '83', '46A', '49', '9', '40', '151', '13', '15B', '61', '140', '79A', '38A', '69', '54A', '150', '150', '56A', '120', '41C', '39X', '68A', '84X', '38', '33X', '26', '44B', '42D', '33D', '40B', '46E', '118', '51D', '15D', '38B', '7B', '41X', '69X', '70D', '77X']
df_GTFS[df_GTFS['LineId'].isin(lineid_list_42routes)].index
df_GTFS=df_GTFS.drop(index=(df_GTFS[df_GTFS['LineId'].isin(lineid_list_42routes)].index))

# #Only keep the LineId where the number of value is greater than 1000
# list = df_GTFS['LineId'].value_counts().loc[lambda x: x>1000].index
# df_GTFS = df_GTFS[df_GTFS['LineId'].isin(list)]

df_GTFS = df_GTFS.rename(columns={'RouteId':'trip_id_new','StartTime':'departure_time','StopSequence':'stop_sequence','StopId':'stop_id'})

df_GTFS

,Time_of_db_update,TripId,trip_id_new,departure_time,StartDate,stop_sequence,stop_id,Arr_delay,Dep_delay,LineId
0,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...,...
641153,1660212000,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,2022-08-11 12:40:00,48,8380B246981,0,0,220
641154,1660212000,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,2022-08-11 12:40:00,12,8440B352481,0,0,603
641155,1660212000,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,2022-08-11 12:45:00,11,8220B1353501,0,0,4
641156,1660212000,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,2022-08-11 12:55:00,48,8380B246981,0,0,220


In [28]:
#Now we will clean the stop_times data in order to intergrate it with the GTFS data

#Load the stop_times file
df_stoptimes = pd.read_csv('../google_transit_combined/stop_times.txt')

#save the unique RouteId into a list
df_GTFS_routeid_list = df_GTFS['trip_id_new'].unique().tolist()

#Filter out the tripID in stoptimes where it appears in GTFS
list_for_concat = []
counter = 0
for i in df_GTFS_routeid_list:
    list_for_concat.append(df_stoptimes[df_stoptimes['trip_id'].str.contains(i)])
    counter += 1
    print('counter = ' , counter, ', finish with ' , i)   
df_stoptimes_new = pd.concat(list_for_concat)

#since two dataset have different tripID, we need to migrate the tripID in GTFS to the df_sotptimes_new
df_stoptimes_new['trip_id_new'] = ''
counter = 0
for i in df_GTFS_routeid_list:
    df_stoptimes_new['trip_id_new'].loc[df_stoptimes_new['trip_id'].str.contains(i)] = i
    counter += 1
    print('counter = ' , counter, ', finish with ' , i)
    
#Cleaning up the irregular time values
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('24','00')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('25','01')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('26','02')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('27','03')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('28','04')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('29','05')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('30','06')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('31','07')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('24','00')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('25','01')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('26','02')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('27','03')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('28','04')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('29','05')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('30','06')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('31','07')

df_stoptimes_new

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


counter =  1 , finish with  10-133-e20-1
counter =  2 , finish with  60-15-d12-1
counter =  3 , finish with  10-740-0-e20-1
counter =  4 , finish with  10-2-e20-1
counter =  5 , finish with  60-C6-d12-1
counter =  6 , finish with  17-126-cm1-1
counter =  7 , finish with  10-722-2-e20-1
counter =  8 , finish with  60-C2-d12-1
counter =  9 , finish with  10-220-e20-1
counter =  10 , finish with  60-65-d12-1
counter =  11 , finish with  60-N4-d12-1
counter =  12 , finish with  10-101-e20-1
counter =  13 , finish with  60-39A-d12-1
counter =  14 , finish with  10-704-e20-1
counter =  15 , finish with  2-331-gad-1
counter =  16 , finish with  60-41-d12-1
counter =  17 , finish with  10-103-e20-1
counter =  18 , finish with  60-C5-d12-1
counter =  19 , finish with  60-C1-d12-1
counter =  20 , finish with  10-23-e20-1
counter =  21 , finish with  10-22-e20-1
counter =  22 , finish with  10-115-e20-1
counter =  23 , finish with  10-32-e20-1
counter =  24 , finish with  10-700-e20-1
counter =  

counter =  196 , finish with  2-761-gad-1
counter =  197 , finish with  10-750-e20-1
counter =  198 , finish with  10-757-e20-1
counter =  199 , finish with  17-125-cm1-1
counter =  200 , finish with  10-721-e20-1
counter =  201 , finish with  10-494-e20-1
counter =  202 , finish with  10-240-e20-1
counter =  203 , finish with  60-X30-d12-1
counter =  204 , finish with  10-774-e20-1
counter =  205 , finish with  2-236-gad-1
counter =  206 , finish with  10-758-e20-1
counter =  207 , finish with  10-104-e20-1
counter =  208 , finish with  10-761-e20-1
counter =  209 , finish with  10-162-e20-1
counter =  210 , finish with  60-16D-d12-1
counter =  211 , finish with  60-X27-d12-1
counter =  212 , finish with  60-X25-d12-1
counter =  213 , finish with  10-795-e20-1
counter =  214 , finish with  10-161-e20-1
counter =  215 , finish with  60-27X-d12-1
counter =  216 , finish with  17-126-6-cm1-1
counter =  217 , finish with  10-241-e20-1
counter =  218 , finish with  10-451-e20-1
counter =  

counter =  386 , finish with  17-126-8-cm1-1
counter =  387 , finish with  10-460-e20-1
counter =  388 , finish with  d12-irl-60041-H-
counter =  389 , finish with  d12-irl-600N4-H-
counter =  390 , finish with  d12-irl-60150-R-
counter =  391 , finish with  10-461-e20-1
counter =  392 , finish with  d12-irl-600C1-R-


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


counter =  1 , finish with  10-133-e20-1
counter =  2 , finish with  60-15-d12-1
counter =  3 , finish with  10-740-0-e20-1
counter =  4 , finish with  10-2-e20-1
counter =  5 , finish with  60-C6-d12-1
counter =  6 , finish with  17-126-cm1-1
counter =  7 , finish with  10-722-2-e20-1
counter =  8 , finish with  60-C2-d12-1
counter =  9 , finish with  10-220-e20-1
counter =  10 , finish with  60-65-d12-1
counter =  11 , finish with  60-N4-d12-1
counter =  12 , finish with  10-101-e20-1
counter =  13 , finish with  60-39A-d12-1
counter =  14 , finish with  10-704-e20-1
counter =  15 , finish with  2-331-gad-1
counter =  16 , finish with  60-41-d12-1
counter =  17 , finish with  10-103-e20-1
counter =  18 , finish with  60-C5-d12-1
counter =  19 , finish with  60-C1-d12-1
counter =  20 , finish with  10-23-e20-1
counter =  21 , finish with  10-22-e20-1
counter =  22 , finish with  10-115-e20-1
counter =  23 , finish with  10-32-e20-1
counter =  24 , finish with  10-700-e20-1
counter =  

counter =  196 , finish with  2-761-gad-1
counter =  197 , finish with  10-750-e20-1
counter =  198 , finish with  10-757-e20-1
counter =  199 , finish with  17-125-cm1-1
counter =  200 , finish with  10-721-e20-1
counter =  201 , finish with  10-494-e20-1
counter =  202 , finish with  10-240-e20-1
counter =  203 , finish with  60-X30-d12-1
counter =  204 , finish with  10-774-e20-1
counter =  205 , finish with  2-236-gad-1
counter =  206 , finish with  10-758-e20-1
counter =  207 , finish with  10-104-e20-1
counter =  208 , finish with  10-761-e20-1
counter =  209 , finish with  10-162-e20-1
counter =  210 , finish with  60-16D-d12-1
counter =  211 , finish with  60-X27-d12-1
counter =  212 , finish with  60-X25-d12-1
counter =  213 , finish with  10-795-e20-1
counter =  214 , finish with  10-161-e20-1
counter =  215 , finish with  60-27X-d12-1
counter =  216 , finish with  17-126-6-cm1-1
counter =  217 , finish with  10-241-e20-1
counter =  218 , finish with  10-451-e20-1
counter =  

counter =  386 , finish with  17-126-8-cm1-1
counter =  387 , finish with  10-460-e20-1
counter =  388 , finish with  d12-irl-60041-H-
counter =  389 , finish with  d12-irl-600N4-H-
counter =  390 , finish with  d12-irl-60150-R-
counter =  391 , finish with  10-461-e20-1
counter =  392 , finish with  d12-irl-600C1-R-


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new
962881,3932872.70.10-133-e20-1.380.I,20:00:00,20:00:00,8350B1355601,1,NaN,0,0,0.00,10-133-e20-1
962882,3932872.70.10-133-e20-1.380.I,20:02:00,20:02:00,8350B106131,2,NaN,0,0,701.08,10-133-e20-1
962883,3932872.70.10-133-e20-1.380.I,20:03:00,20:03:00,8350B1060701,3,NaN,0,0,1144.17,10-133-e20-1
962884,3932872.70.10-133-e20-1.380.I,20:08:00,20:08:00,8350B1355501,4,NaN,0,0,2577.73,10-133-e20-1
962885,3932872.70.10-133-e20-1.380.I,20:10:00,20:10:00,8350B1060801,5,NaN,0,0,2971.39,10-133-e20-1
...,...,...,...,...,...,...,...,...,...,...
1605666,3924636.50.10-461-e20-1.1796.O,07:43:00,07:43:00,8500B5981701,3,NaN,0,0,18002.14,10-461-e20-1
1605667,3924636.50.10-461-e20-1.1796.O,07:47:00,07:47:00,8500B555811,4,NaN,0,0,22319.30,10-461-e20-1
1605668,3924636.50.10-461-e20-1.1796.O,07:50:06,07:50:06,8500B5986201,5,NaN,0,0,26377.64,10-461-e20-1
1605669,3924636.50.10-461-e20-1.1796.O,07:54:00,07:54:00,8500B555611,6,NaN,0,0,30947.01,10-461-e20-1


In [31]:
df_stoptimes_new.to_csv('df_stoptimes_new',index=False)

In [29]:
#save all the line id in a list
# lineid_list = df_GTFS['LineId'].unique().tolist()
# lineid_list.remove('15')
# lineid_list.remove('133')
# lineid_list.remove('740')
# lineid_list.remove('126')
# lineid_list.remove('722')
# lineid_list.remove('C2')
# lineid_list

lineid_list = ['451','155','6','C1','C2','C3','C4','H1','H2','H3','L54','N4']

In [30]:
weather = pd.read_csv('hly175.csv')
weather['date']=weather['date'].astype('datetime64')

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# count = 1

# start_time = time.time()

# # looping over each lineid
# for lineid in lineid_list:
    
#     now = time.time()
#     timer = now - start_time
#     print("Now working on line:", lineid, "number", count, "in the sequence. Time since start", timer)
#     count += 1
#     data = lineid
#     %store data
#     del data
#     %run ./GTFS_DataProcessing_Child.ipynb

In [32]:
lineid_list_extra = ['174','175','331','332','65B','751','761','951','952','C5','C6','L53','L58','L59','X25','X27','X28','X30','X31']

In [33]:
count = 1

start_time = time.time()

# looping over each lineid
for lineid in lineid_list_extra:
    
    now = time.time()
    timer = now - start_time
    print("Now working on line:", lineid, "number", count, "in the sequence. Time since start", timer)
    count += 1
    data = lineid
    %store data
    del data
    %run ./GTFS_DataProcessing_Child.ipynb

Now working on line: 174 number 1 in the sequence. Time since start 0.00022292137145996094
Stored 'data' (str)


<ipython-input-33-80418484b86a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptime_15['StartDate'] = ''
<ipython-input-33-80418484b86a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GTFS_15['StartDate']=df_GTFS_15['StartDate'].astype(str)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

<ipython-input-33-80418484b86a>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptimes_merged['actual_arrival_time'][i] = df_stoptimes_merged['arrival_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Arr_delay'][i])
<ipython-input-33-80418484b86a>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptimes_merged['actual_departure_time'][i] = df_stoptimes_merged['departure_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Dep_delay'][i])
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

Finished with calculating actual depart time and actual arrive time
Only one headsign or Nan value in headsign column.
Now working on line: 175 number 2 in the sequence. Time since start 1.383145809173584
Stored 'data' (str)


<ipython-input-33-80418484b86a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptime_15['StartDate'] = ''
<ipython-input-33-80418484b86a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GTFS_15['StartDate']=df_GTFS_15['StartDate'].astype(str)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

...

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
<ipython-input-33-80418484b86a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

Finished with calculating actual depart time and actual arrive time
Length of headsign:  2
Now working on line: X31 number 19 in the sequence. Time since start 37.13502788543701
Stored 'data' (str)
Finished with calculating actual depart time and actual arrive time

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
<ipython-input-33-80418484b86a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat


Length of headsign:  2


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [ ]:
#Here we test loading the pickle file 

In [34]:
dictionary = {'stop_sequence':15,'hour_of_day':8,'day_of_week':5,'month_of_year':8,'temp':23,'rain':0}

In [35]:
var = pd.DataFrame([dictionary])
var

,stop_sequence,hour_of_day,day_of_week,month_of_year,temp,rain
0,15,8,5,8,23,0


In [37]:
#Load the model
line25_model = pickle.load(open('ModelResult_RandomForest_GTFS/PickleFiles_175.pkl', 'rb'))

In [38]:
line25_model.predict(X_test_25)

NameError: name 'X_test_25' is not defined

In [39]:
line25_model.predict(var)

array([3558.])